In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Ovarian_Cancer"
cohort = "GSE130402"

# Input paths
in_trait_dir = "../../input/GEO/Ovarian_Cancer"
in_cohort_dir = "../../input/GEO/Ovarian_Cancer/GSE130402"

# Output paths
out_data_file = "../../output/preprocess/Ovarian_Cancer/GSE130402.csv"
out_gene_data_file = "../../output/preprocess/Ovarian_Cancer/gene_data/GSE130402.csv"
out_clinical_data_file = "../../output/preprocess/Ovarian_Cancer/clinical_data/GSE130402.csv"
json_path = "../../output/preprocess/Ovarian_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE130402_family.soft.gz', 'GSE130402_series_matrix.txt.gz']
SOFT file: ../../input/GEO/Ovarian_Cancer/GSE130402/GSE130402_family.soft.gz
Matrix file: ../../input/GEO/Ovarian_Cancer/GSE130402/GSE130402_series_matrix.txt.gz
Background Information:
!Series_title	"MiRNA-mediated induction of mesenchymal-to-epithelial transition (MET) between cancer cell types is significantly modulated by inter-cellular molecular variability"
!Series_summary	"Recent years have witnessed a dramatic increase in our appreciation of the contribution of microRNAs (miRNAs) to cancer onset and progression. As a consequence, there has been growing interest in the development of miRNAs not only as diagnostic biomarkers of cancer but also as a promising new class of therapeutic agents. Over the last several years, our laboratory has focused on analysis of the molecular processes underlying the ability of individual miRNAs to induce mesenchymal-to-epithelial transition (MET) particularly in

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import json
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression microarray data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability 
# Looking at the sample characteristics dictionary:
# - For trait (Ovarian Cancer): The dataset includes both ovarian cancer (HEY, SKOV3) and prostate cancer (PC3) cell lines
# - We can use the 'cell line' in row 0 to distinguish between ovarian and non-ovarian cancer
trait_row = 0

# Age and gender are not provided in this cell line dataset
age_row = None
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value: str) -> int:
    """Convert cell line information to binary trait (Ovarian Cancer = 1, Other = 0)"""
    if value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Ovarian cancer cell lines are HEY and SKOV3
    if 'HEY' in value or 'SKOV3' in value:
        return 1
    else:
        # PC3 is a prostate cancer cell line
        return 0

def convert_age(value: str) -> Optional[float]:
    """Convert age information to float."""
    # Age is not available in this dataset
    return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender information to binary (0 for female, 1 for male)."""
    # Gender is not available in this dataset
    return None

# 3. Save Metadata
# Conduct initial filtering based on trait and gene data availability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # For this dataset, let's create a DataFrame from the sample characteristics dictionary
    # The sample characteristics are shown in the previous output
    samples = ['HEY cells', 'SKOV3 cells', 'PC3 cells']
    cell_lines = [f"cell line: {sample}" for sample in samples]
    
    # Create a DataFrame in the format expected by geo_select_clinical_features
    clinical_data = pd.DataFrame({
        'ID_REF': samples,
        0: cell_lines  # corresponds to trait_row=0
    })
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the data
    preview = preview_df(selected_clinical_df)
    print("Clinical data preview:")
    print(preview)
    
    # Save clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical data preview:
{'ID_REF': [1.0], 0: [1.0]}
Clinical data saved to ../../output/preprocess/Ovarian_Cancer/clinical_data/GSE130402.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.



Gene data extraction result:
Number of rows: 54675
First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers appear to be probe IDs from an Affymetrix microarray
# The format '1007_s_at', '1053_at', etc. is characteristic of Affymetrix probe identifiers
# These are not human gene symbols and will need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the columns for gene identifier and gene symbol
# From the gene annotation preview, we can see:
# - 'ID' column contains identifiers like '1007_s_at' which match our gene expression data
# - 'Gene Symbol' column contains human gene symbols like 'DDR1 /// MIR4640'

# Extract gene expression data from matrix file
gene_data = get_genetic_data(matrix_file)
print(f"Gene expression data loaded with {len(gene_data)} rows and {len(gene_data.columns)} columns")

# 2. Create gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'Gene Symbol')
print(f"Gene mapping created with {len(gene_mapping)} rows")
print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Convert probe-level measurements to gene expression data
print("\nConverting probe measurements to gene expression data...")
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data created with {len(gene_data)} rows and {len(gene_data.columns)} columns")
print("Gene expression data preview (first 5 genes, first 5 samples):")
if len(gene_data) > 0:
    print(gene_data.iloc[:5, :5])
else:
    print("No gene data was produced. This might indicate an issue with the mapping.")

# 4. Normalize gene symbols to ensure consistency
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"After normalization: Gene expression data has {len(gene_data)} rows")


Gene expression data loaded with 54675 rows and 45 columns


Gene mapping created with 45782 rows
Gene mapping preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'Gene': ['DDR1 /// MIR4640', 'RFC2', 'HSPA6', 'PAX8', 'GUCA1A']}

Converting probe measurements to gene expression data...


Gene expression data created with 21278 rows and 45 columns
Gene expression data preview (first 5 genes, first 5 samples):
          GSM3737598  GSM3737599  GSM3737600  GSM3737601  GSM3737602
Gene                                                                
A1BG         5.00532     4.44208     4.80766     3.71290     3.10398
A1BG-AS1     5.68344     5.49375     5.88851     4.96129     5.02876
A1CF         7.36688     7.05463     6.99478     4.69005     5.03059
A2M          8.06830     7.77643     8.02527     7.44513     7.16483
A2M-AS1      3.14186     3.37572     3.67574     6.97224     7.07981
After normalization: Gene expression data has 19845 rows


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols using the NCBI Gene database synonym information
print("Normalizing gene symbols using NCBI synonym information...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Number of genes before normalization: {len(gene_data)}")
print(f"Number of genes after normalization: {len(normalized_gene_data)}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# 2. Since we determined in step 2 that there's no usable trait variation 
# (all samples are cancer cases with no controls), we'll create a clinical dataframe
# but note this limitation

# Create a clinical dataframe with the trait (Ovarian_Cancer)
sample_ids = normalized_gene_data.columns.tolist()
print(f"Sample IDs from gene data: {len(sample_ids)} samples")

# Create clinical dataframe, but note that all samples have the same trait value
clinical_df = pd.DataFrame(index=[trait], columns=sample_ids)
clinical_df.loc[trait] = 1  # All samples are ovarian cancer tumors

print(f"Clinical data shape: {clinical_df.shape}")

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
print(f"Shape of linked data: {linked_data.shape}")

# 4. Handle missing values in the linked data
print("Handling missing values...")
linked_data_cleaned = handle_missing_values(linked_data, trait)
print(f"Shape of linked data after handling missing values: {linked_data_cleaned.shape}")

# 5. Check if the trait and demographic features are biased
print("Checking for bias in features...")
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_cleaned, trait)

# 6. Validate the dataset and save cohort information
note = "Dataset contains expression data for ovarian cancer tumors. All samples are tumor samples with no controls, so trait bias is expected and the dataset is not suitable for case-control analysis."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True, 
    is_biased=is_trait_biased,
    df=unbiased_linked_data,
    note=note
)

# 7. Save the linked data if it's usable (though we expect it won't be due to trait bias)
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Saved processed linked data to {out_data_file}")
else:
    print("Dataset validation failed due to trait bias. Final linked data not saved.")

Normalizing gene symbols using NCBI synonym information...


Number of genes before normalization: 19845
Number of genes after normalization: 19845


Normalized gene expression data saved to ../../output/preprocess/Ovarian_Cancer/gene_data/GSE130402.csv
Sample IDs from gene data: 45 samples
Clinical data shape: (1, 45)
Clinical data saved to ../../output/preprocess/Ovarian_Cancer/clinical_data/GSE130402.csv
Shape of linked data: (45, 19846)
Handling missing values...


/media/techt/DATA/GenoAgent/tools/preprocess.py:455: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[gene_cols] = df[gene_cols].fillna(df[gene_cols].mean())


Shape of linked data after handling missing values: (45, 19846)
Checking for bias in features...
Quartiles for 'Ovarian_Cancer':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1
Max: 1
The distribution of the feature 'Ovarian_Cancer' in this dataset is severely biased.

Dataset validation failed due to trait bias. Final linked data not saved.
